In [5]:
import pandas as pd
import scanpy as sc
import anndata as ad
import pymn
import matplotlib
import scipy.stats
import itertools
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sc.settings.verbosity = 3

In [1]:
def read_in_sunil_formated_data(
    file_name,
    file_location="/data/passala/Collaborator_Data/Sunil_Ken_root_Collab/2024_update/Root_seperated_types",
    counts=False,
):
    """Take in data from Sunil - formatted as 3 files with the same prefix

    Args:
        file_name (str): prefix of the 3 file
        file_location (str, optional): folder files are in. Defaults to '/data/passala/Collaborator_Data/Sunil_Ken_root_Collab/Project_2_across_roots'.
        counts (boolean,optional): Whether to return the raw counts or the corrected. When False returns corrected data. Defaults to False
    Raises:
        Exception: _description_
        Exception: _description_
        Exception: _description_

    Returns:
        anndata: Either the counts or the corrected anndata
    """
    base_file_name = file_name
    file_location = file_location
    counts_file = file_location + "/" + base_file_name + "_counts.csv"
    counts_pd = pd.read_csv(counts_file, index_col=0)
    counts_pd = counts_pd.T

    corrected_file = file_location + "/" + base_file_name + "_data.csv"
    corrected_pd = pd.read_csv(corrected_file, index_col=0)
    corrected_pd = corrected_pd.T

    meta_file = file_location + "/" + base_file_name + "_meta.csv"
    meta_pd = pd.read_csv(meta_file, index_col=0)

    if all(meta_pd.index == counts_pd.index) == False:
        raise Exception("Meta data doesn't match counts index")
    if all(meta_pd.index == corrected_pd.index) == False:
        raise Exception("Meta data doesn't match corrected index")
    if all(counts_pd.index == corrected_pd.index) == False:
        raise Exception("Counts doesn't match corrected data")

    counts_anndata = ad.AnnData(X=counts_pd.values)
    counts_anndata.var.index = counts_pd.columns
    counts_anndata.obs = meta_pd

    corrected_anndata = ad.AnnData(X=corrected_pd.values)
    corrected_anndata.var.index = corrected_pd.columns
    corrected_anndata.obs = meta_pd

    if counts == False:
        return corrected_anndata
    if counts == True:
        return counts_anndata

In [2]:
list_of_root_types = ['crown_rename','primary_lat_rename','primary_rename','seminal_lat_rename','seminal_rename']


In [6]:
root_data_dictionary = {}
for name_root in list_of_root_types:
    current_root = read_in_sunil_formated_data(file_name= name_root,)
    root_data_dictionary[name_root] = current_root

/tmp/ipykernel_94794/4197476692.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  counts_anndata = ad.AnnData(X=counts_pd.values)
/tmp/ipykernel_94794/4197476692.py:44: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  corrected_anndata = ad.AnnData(X=corrected_pd.values)
/tmp/ipykernel_94794/4197476692.py:40: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  counts_anndata = ad.AnnData(X=co

In [9]:
root_data_dictionary['crown_rename'].obs['CellType'].unique()

array(['10_epidermis', '3_pericycle_stele', '13_columella?', '4_stele',
       '1_phloem', '0_pericycle', '2_pericycle_stele', '7_G2M/cortex',
       '5_cortex_epidermis', '6_endodermis', '15_cortex_epidermis',
       '16_phloem', '9_xylem/stele', '11_pericycle_stele',
       '8_cortex_epidermis', '14_phloem_stele', '12_epidermis'],
      dtype=object)

In [10]:
root_data_dictionary['primary_rename'].obs['CellType']

Primary_AAACCCAAGACTAAGT       14_epidermis_cortex
Primary_AAACCCAAGTGGCCTC               4_pericycle
Primary_AAACCCACAAGGTCAG       14_epidermis_cortex
Primary_AAACCCACACGAAGAC                  0_cortex
Primary_AAACCCATCAATCTCT    13_epidermis_columella
                                     ...          
Primary_TTTGGTTCACTTCAAG        7_epidermis_cortex
Primary_TTTGGTTCATCTCATT              12_epidermis
Primary_TTTGTTGAGGCCTTCG             11_endodermis
Primary_TTTGTTGAGTCACTGT                  8_cortex
Primary_TTTGTTGCATGTGCTA        7_epidermis_cortex
Name: CellType, Length: 4790, dtype: object

In [22]:
def process_sunil_data_to_get_a_coexpression_network(sc_file,save_name):
    sc.pp.filter_cells(sc_file, min_genes=200)
    sc.pp.filter_genes(sc_file, min_cells=3)
    sc.pp.highly_variable_genes(sc_file, min_mean=0.125, max_mean=4, min_disp=0.25)
    sc.tl.pca(sc_file, svd_solver='arpack', random_state=194)
    sc.pp.neighbors(sc_file, n_neighbors=12, n_pcs=50)
    sc.tl.umap(sc_file, random_state = 53)
    sc.tl.leiden(sc_file,resolution = 150, random_state = 203)
    psuedobulk_df = pd.DataFrame(index = sc_file.var_names)## Make a base dataframe index we will add stuff on to later
    all_samples = list(sc_file.obs.leiden.unique())  ## get list of clusters to loop through
    
    for batch_type in all_samples:

        ## Read in the Names so our code is easy to understand
        current_cluster = batch_type

        ## Calculate the Psuedobulked mean
        cells_matching_batch_and_cluster = sc_file[sc_file.obs['leiden'] == current_cluster ]
        mean_of_genes = cells_matching_batch_and_cluster.X.mean(axis = 0).tolist()


        name_of_combo = current_cluster
        psuedobulk_df[name_of_combo] = mean_of_genes
    import numpy as np
    import scipy.stats as sci

    rank_test_py_exp = sci.rankdata(psuedobulk_df, method = 'average', axis = 1)                #Row ranks
    rank_test_py_exp = rank_test_py_exp - rank_test_py_exp.mean(axis = 1)[1]               #Center each gene, subtract mean rank
    rank_test_py_exp_2 = np.square(rank_test_py_exp)                                       #Square
    rank_test_py_exp = rank_test_py_exp /np.sqrt(rank_test_py_exp_2.sum(axis = 1))[:,None] #divide by sqrt(rowSums)
    cr_python = np.dot(rank_test_py_exp, rank_test_py_exp.T)
    corr_results = pd.DataFrame(columns = psuedobulk_df.index, index = psuedobulk_df.index, data = cr_python)
    corr_results.to_csv(f'/data/passala/Generated_Tables/Sunil_root_shoot_project/{save_name}_corr_network.csv')

In [20]:
root_data_dictionary['primary_rename']

AnnData object with n_obs × n_vars = 4790 × 25086
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.0.5', 'seurat_clusters', 'MeristemScore', 'DifferentiationScore', 'CellType', 'n_genes', 'leiden'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'pca', 'neighbors', 'umap', 'leiden'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [24]:
for root_type in list_of_root_types:
    process_sunil_data_to_get_a_coexpression_network(root_data_dictionary[root_type],root_type)
    

filtered out 112 genes that are detected in less than 3 cells
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:03)
running Leiden clustering
    finished: found 665 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

filtered out 115 genes that are detected in less than 3 cells
extracting highly variable genes
    finished (0:00:01)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:09)
running Leiden clustering
    finished: found 943 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:07)
running Leiden clustering
    finished: found 794 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

filtered out 107 genes that are detected in less than 3 cells
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:08)
running Leiden clustering
    finished: found 877 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

filtered out 546 genes that are detected in less than 3 cells
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:09)
running Leiden clustering
    finished: found 846 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  psuedobulk_df[name_of_combo] = mean_of_genes
/tmp/ipykernel_94794/1452321372.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

In [19]:
root_data_dictionary['primary_rename'].obs['leiden'].value_counts().head(20)

0     13
6     12
1     12
10    12
8     12
7     12
9     12
5     12
4     12
3     12
2     12
20    11
28    11
27    11
26    11
25    11
24    11
22    11
21    11
23    11
Name: leiden, dtype: int64